In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()

from PyCRO_SL import *
from PyCRO_SL.CRO_SL import CRO_SL
from PyCRO_SL.AbsObjetiveFunc import AbsObjetiveFunc
from PyCRO_SL.SubstrateInt import SubstrateInt
from PyCRO_SL.SubstrateReal import SubstrateReal
import random

In [11]:
class HydroProblemSemidist(AbsObjetiveFunc):
    def __init__(self, basin_df, rscript_name="exec_optim_semidist.R", metric="MSE", model_used=0, basin_code=-1):
        # Defining the R script and loading the instance in Python
        r = robjects.r["source"]
        # (rscript_name)

        inf_lim = np.array([1e-4, 10, 0, 1e-4, 1e-4, 2, -6])
        sup_lim = np.array([1, 2000, 1, 100, 1, 11, 1])
        self.size = 7

        self.metric = metric
        self.model_used = model_used
        self.basin_df = basin_df
        self.basin_code = basin_code

        opt = "min"
        if metric in ["KGE", "NSE"]:
            opt = "max"

        # Loading the function we have defined in R.
        self.get_basin_q = robjects.globalenv["get_basin_q"]
        self.exec_function_r = robjects.globalenv["eval_basin"]
        super().__init__(self.size, opt, sup_lim, inf_lim)

    def objetive(self, solution):
        agg_q = {}
        result_q = {}

        prev_q = 0
        for idx in self.basin_df.index:
            basin_code = self.basin_df["code"][idx]
            codedown = self.basin_df["codedown"][idx]

            prev_q = 0
            if basin_code in agg_q:
                prev_q = agg_q[basin_code]

            # agg_q[codedown] += self.get_basin_q(model, solution, basin_code, prev_q)
            result_q[basin_code] = self.get_basin_q(model, solution, basin_code, prev_q)
            if codedown not in agg_q:
                agg_q[codedown] = 0
            agg_q[codedown] += result_q[basin_code]

        metrics = self.exec_function_r(result_q[self.basin_code], self.basin_code)
        if self.metric == "MSE":
            return float(metrics[0])
        elif self.metric == "RMSE":
            return float(metrics[1])
        elif self.metric == "NSE":
            return float(metrics[3])
        elif self.metric == "R2":
            return abs(1 - float(metrics[4]))
        elif self.metric == "KGE":
            return float(metrics[5])

    def random_solution(self):
        return (self.sup_lim - self.inf_lim) * np.random.random(self.size) + self.inf_lim

    def check_bounds(self, solution):
        return solution.clip(self.inf_lim, self.sup_lim)


HydroProblemSemidist("exec_optim_semidist.R", "NSE", 0)

In [8]:
substrates_real = [
    SubstrateReal("Gauss", {"F": 0.1}),
    SubstrateReal("DE/best/2", {"F": 0.7, "Cr": 0.9}),
    SubstrateReal("BLXalpha", {"Cr": 0.35}),
    SubstrateReal("Firefly", {"a": 0.7, "b": 1, "d": 0.95, "g": 10}),
    SubstrateReal("Perm", {"Cr": 4 / 7}),
]

params = {
    # "popSize": 120,
    "popSize": 2,
    "rho": 0.6,
    "Fb": 0.98,
    "Fd": 0.15,
    "Pd": 0.99,
    "k": 3,
    "K": 10,
    "group_subs": True,
    "stop_cond": "ngen",
    "time_limit": 400.0,
    "Ngen": 1,
    # "Ngen": 100,
    "Neval": 3e4,
    "fit_target": 1000,
    "verbose": True,
    "v_timer": 1,
    "dynamic": True,
    "dyn_method": "fitness",
    "dyn_metric": "best",
    "dyn_steps": 500,
    "prob_amp": 0.015,
}

In [13]:
model = 2
target = "NSE"

rscript_name = "exec_optim_semidist.R"

r = robjects.r
r["source"](rscript_name)
get_basin_q = robjects.globalenv["get_basin_q"]

basins = pd.read_csv("basins.txt")
basins.sort_values(by=["order"])

objfunc = HydroProblemSemidist(basins, rscript_name, target, model, 3005)
c = CRO_SL(objfunc, substrates_real, params)

print(f"Optimizing basin with code {basin_code}.\n\n")

c.safe_optimize()

# agg_q = {}
# basin_params = {}

# prev_q = 0
# for idx in basins.index:
#     basin_code = basins["code"][idx]
#     codedown = basins["codedown"][idx]

#     prev_q = 0
#     if basin_code in agg_q:
#         prev_q = agg_q[basin_code]

#     objfunc = HydroProblemSemidist("exec_optim_semidist.R", "NSE", 0)
#     c = CRO_SL(objfunc, substrates_real, params)

#     print(f"Optimizing basin with code {basin_code}.\n\n")

#     c.safe_optimize()
#     basin_params[basin_code] = c.best_solution()

#     if codedown not in agg_q:
#         agg_q[codedown] = 0

#     agg_q[codedown] += get_basin_q(model, c.best_solution()[0], basin_code, prev_q)

#     print("agg_q:", agg_q)
#     print("basin:", basin[:][idx])

Rows: 4 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): code, order, codedown, supha

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Optimizing basin with code 3001.


[ 3.26267415e+03  5.71198230e+01  1.37100000e+02 -3.04693500e+00
  2.30821000e-01 -4.69585000e-01]
[ 3.26267415e+03  5.71198230e+01  1.37100000e+02 -3.04693500e+00
  2.30821000e-01 -4.69585000e-01]
[ 3.26267415e+03  5.71198230e+01  1.37100000e+02 -3.04693500e+00
  2.30821000e-01 -4.69585000e-01]
Time Spent 13.46s:
	Generation: 1
	Best fitness: -3.046935
	Evaluations of fitness: 3
	Substrate probability:
		Gauss:    0.2
		DE/best/2:0.2
		BLXalpha: 0.2
		Firefly:  0.2
		Perm:     0.2



(array([ 1.39732610e-01,  1.48888156e+03,  2.58168254e-01,  9.70876390e+01,
         4.67081482e-03,  7.02761415e+00, -3.45334615e+00]),
 -3.046935)

In [65]:
# result = []
# for i in basins["code"]:
#     result += [basin_params[i][0]]
# result = np.array(result)
result = c.best_solution()[0]
np.savetxt("config_nooptim.csv", [result], delimiter=",")

In [66]:
c.best_solution()[1]

-2.364329